# KD Tree for Bottleneck TSP

In [1]:
%load_ext autoreload
%autoreload 2

## Load Spotify Data

In [2]:
import tekore as tk
from smoothify.features import construct_features
import numpy as np

In [3]:
ACCESS_TOKEN = "BQDgc0ursrbB6EPOmR-b6p8K7N5JrSk9I8DdqAQo9V4dmhPTkHkNbeU2TYAsLTzz-5i4WRbirohiIc2MFVANyLdndKbKMLF567oJEtejqXHaa9aeror8DCpTCopUJdotTgTk145DLGKIBUy-PuSPkOm5sXYB5zRzu1fAcJkvCVK61kPdBkOFm9dpiusiz6bU3G7e2Mjdmf6M61KTgJG8ImN-6xZG8RDDwa2KkxUhtZAkZcYR"
spotify = tk.Spotify(ACCESS_TOKEN, asynchronous=True, max_limits_on=True, chunked_on=True)
current_user = await spotify.current_user()

### Fetch tracks from current user's library

In [4]:
playlist_name = f"{current_user.display_name}'s Library"
saved_tracks_page = await spotify.saved_tracks()
track_list = [track.track async for track in spotify.all_items(saved_tracks_page)]

### Fetch tracks from a playlist

In [5]:
PLAYLIST_ID = "5TNvDYA16DHMKkBQaINAid"
playlist = await spotify.playlist(PLAYLIST_ID)
playlist_name = playlist.name
track_list = [track.track async for track in spotify.all_items(playlist.tracks)]

### Query Spotify API for audio features

In [6]:
# Extract track IDs so we can fetch additional info
track_id_list = [track.id for track in track_list]
# Get track audio features
track_features_list = await spotify.tracks_audio_features(track_id_list)

### Construct & normalize audio features

In [7]:
features_df = construct_features(audio_features_list=track_features_list)
features_df.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,-0.519845,1.067537,1.478168,0.038817,-0.281472,-0.377442,0.611489,0.624322,-0.025086
1,-1.577793,0.331305,0.403142,-0.367696,-0.595380,-0.377410,0.617376,-0.975399,-0.328454
2,-1.024653,0.849394,0.585285,-0.459390,-0.890693,-0.377442,-0.189151,0.067897,2.061538
3,1.268464,-1.495643,-1.065699,-0.543444,0.947858,-0.377254,-0.960945,0.677315,-0.205916
4,0.044036,-0.604892,-0.067566,-0.220984,-0.811513,-0.377442,2.212769,-1.088009,-0.203277


In [8]:
points = np.array(features_df)

## Compute the best path through the points

In [9]:
import random
from uuid import uuid4
from functools import partial
from typing import List, Tuple

import pandas as pd
import scipy.spatial
from scipy.spatial import cKDTree
from joblib import Parallel, delayed

import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
sns.set_theme()

In [10]:
from smoothify.optim.bottleneck_tsp import KDTreeBottleneckTSP

In [11]:
optimizer = KDTreeBottleneckTSP(points=points)
results = optimizer.get_best_path()
best_path = results.best_path
max_dist = results.min_max_dist
print(f"Max edge length: {max_dist}")

  0%|                                                                                                                                                                                | 0/10 [00:00<?, ?it/s]

[ProgressParallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:01<00:00,  6.81it/s]

[ProgressParallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.5s finished
Max edge length: 4.745401329187675


### Create a smoothified playlist

In [12]:
# Create new playlist
NEW_PLAYLIST_NAME = f"{playlist.name} (but smoother)"
new_playlist = await spotify.playlist_create(playlist.owner.id, NEW_PLAYLIST_NAME, public=playlist.public)
await spotify.playlist_change_details(new_playlist.id, collaborative=playlist.collaborative)

# Add tracks to new playlist
smoothified_track_uris = [track_list[node_idx].uri for node_idx in best_path]
await spotify.playlist_add(new_playlist.id, smoothified_track_uris)

'Myw2YzcwOWMzNjJlYWZlNDA2N2JlZTc0ZDg4OGU5NzY0YWFkYTRlODUx'